In [48]:
#loading necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import pickle
#import pycountry

from wordcloud import WordCloud
from langdetect import detect

# #gensim
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

# libraries for NLP
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import SnowballStemmer, PorterStemmer, LancasterStemmer
from nltk.stem import WordNetLemmatizer


# importing libraries for utility from sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV

# importing libraries for modelling form sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# libraries to check model metrics from sklearn
from sklearn.metrics import confusion_matrix, f1_score, classification_report,accuracy_score

# Enable interactive visualisations in Jupyter
from ipywidgets import interact, interact_manual, widgets
import plotly.express as px

# Suppressing unnwarranted warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
pip install vaderSentiment

     ------------------------------------ 126.0/126.0 kB 389.6 kB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install langdetect

     -------------------------------------- 981.5/981.5 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=0bd619b67f21c3bf25385aee890df7a33920928715e47104747a472035ee88b2
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\13\c7\b0\79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [49]:
#importing packages for tweet scraping
from datetime import date,timedelta,datetime
import snscrape.modules.twitter as sntwitter
import pandas as pd
import time
# import nltk
from nltk.corpus import stopwords
import re
# scheduler
import schedule
import os.path
# aws
# nltk.download('stopwords')
stopwords = stopwords.words('english')

In [4]:
pip install schedule


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Tweet Scraping

In [50]:
def fetch_all_tweets():
    ''' Fetch all tweets from 2017 september based on the keywords provided, it returns a dataframe'''
    tweets_list2 = []
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'(shell affordable energy) OR (shell clean energy) OR (shell affordable clean energy)  OR (shell decent work) OR (shell economic growth) OR (shell decent work economic growth) OR (shell industry innovation infrastructure) OR (shell innovation) OR (shell industry innovation) OR (shell responsible consumption) OR (shell responsible production) OR (shell climate action) OR (shell parternship) OR (shell partnership for goals)  lang:en since:2017-09-01 until:{date.today()}').get_items()):
        print(f"Tweeets {tweet.date}  {tweet.id}")
        tweets_list2.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username,tweet.user.verified,tweet.user.location,tweet.replyCount,tweet.retweetCount,tweet.likeCount,tweet.quoteCount])
        tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet_Id', 'Text', 'Username','Verified','Location','Reply_Count','Retweet_Count','Like_Count','Quote_Count'])
    return tweets_df2

In [ ]:
def fetch_sectioned_tweets(tweets_df):
    '''Fetch tweets added since the time the last data was fetched and return a dataframe'''
    # read the already existing csv file
    df = tweets_df
    # Check maximum date
    d = datetime.strptime(df['Datetime'].agg('max')[0:-6], "%Y-%m-%d %H:%M:%S")
    max_date = int(time.mktime(d.timetuple())+1)
    now_time = int((datetime.now()+timedelta(days=1)).timestamp())
    sectioned_tweet_list = []
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'(shell affordable energy) OR (shell clean energy) OR (shell affordable clean energy)  OR (shell decent work) OR (shell economic growth) OR (shell decent work economic growth) OR (shell industry innovation infrastructure) OR (shell innovation) OR (shell industry innovation) OR (shell responsible consumption) OR (shell responsible production) OR (shell climate action) OR (shell parternship) OR (shell partnership for goals) lang:en  since_time:{max_date} until_time:{now_time}').get_items()):
        sectioned_tweet_list.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username,tweet.user.verified,tweet.user.location,tweet.replyCount,tweet.retweetCount,tweet.likeCount,tweet.quoteCount])
        sectioned_tweets_df = pd.DataFrame(sectioned_tweet_list, columns=['Datetime', 'Tweet_Id', 'Text', 'Username','Verified','Location','Reply_Count','Retweet_Count','Like_Count','Quote_Count'])

    return sectioned_tweets_df


In [ ]:
def clean_data(tweet_df):
    '''Clean data: i. drop duplicates ii. remove punctuations iii. Remove characters less than 3, it returns a dataframe '''
    # remove duplicates
    bul = tweet_df.duplicated(subset='Tweet_Id', keep='first' )
    tweets_df2=tweet_df[~bul]
    # Remove characters less than 3 and all mentions
    tweets_df2['text_without_punctuations']= tweets_df2['Text'].apply(lambda x: " ".join ([w for w in x.split() if len (w)>1 and '@' not in w[0:3] and 'amp' not in w]))
     # remove punctuations  
    pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    subs_url = " "
    tweets_df2['text_without_punctuations'] = tweets_df2['text_without_punctuations'].replace(to_replace = pattern_url, 
                                          value = subs_url, regex = True)
    tweets_df2['text_without_punctuations']= tweets_df2['text_without_punctuations'].str.replace("[^'a-zA-Z0–9]", ' ')
    return tweets_df2

In [ ]:
# Functions to be written 
# 1. fetch and create dataframe
# 2. Compare already existing dataframe to see if there is any additional records
# 3. drop duplicates
# 4. remove punctuations
# 5. tokenize
# 6. remove stop words

### Importing Dataset

In [51]:
def import_dataset(path: str) -> pd.DataFrame:
    """
    Data read_csvtion, importing `csv` file only
    Args:
        path: string like `file` address in the directory
    Return:
        df: (Dataframe) output imported csv file as pandas Dataframe
    """
    data = pd.read_csv(path)
    data = data[['Datetime', 'Text', 'Username','Verified', 'Location', 'Reply_Count','Retweet_Count', 'Like_Count', 
                 'Quote_Count']] # Select columns to work with
    data = data.drop_duplicates( subset = 'Text') # drop duplicates tweets
    #data = data[data['Text'].apply(detect).eq('en')] # filter non-english tweets
    
    
    return data

In [52]:
data = import_dataset('tweets_2.csv')

In [53]:
data.head(2)

,Datetime,Text,Username,Verified,Location,Reply_Count,Retweet_Count,Like_Count,Quote_Count
0,2022-09-20 12:28:48+00:00,or we could invest in renewables and not worry...,bertytrouser,False,"Wales, United Kingdom",0,0,0,0
1,2022-09-20 11:46:32+00:00,"Sabine Brink, Blockchain CoE Lead at Shell giv...",Blockchain_Expo,False,Global,1,3,10,1


# Data cleaning

In [59]:
def data_preprocessing(df):
    '''
    Function takes in the whole dataframe and carries out the following preprocessing steps:
    
    1. General text cleaning
    2. Part of Speech tagging
    3. Lemmatization
    
    Then return the dataframe
    '''
    def clean_tweet(tweet):
        '''
        tweet: String
               Input Data
        tweet: String
               Output Data

        func: Removes hashtag symbol in front of a word
              Replace URLs with a space in the message
              Replace ticker symbols with space. The ticker symbols are any stock symbol that starts with $.
              Replace  usernames with space. The usernames are any word that starts with @.
              Replace everything not a letter or apostrophe with space
              Remove single letter words
              filter all the non-alphabetic words, then join them again

        '''
    
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

        tweet = re.sub('\$[a-zA-Z0-9]*', ' ', tweet)
        tweet = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', tweet)
        tweet = re.sub('\@[a-zA-Z0-9]*', ' ', tweet)
        tweet = re.sub('[^a-zA-Z\']', ' ', tweet)
        tweet = re.sub(r'\s+', " ", tweet)
        tweet = ' '.join( [w for w in tweet.split() if len(w)>1] )
    
        return tweet
    
    def token_stop_pos(text):
        '''
        Maps the part of speech to words in sentences giving consideration to words that are nouns, verbs, 
        adjectives and adverbs
        '''
        pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
        tags = pos_tag(word_tokenize(text))
        newlist = []
        for word, tag in tags:
            if word.lower() not in set(stopwords):
                newlist.append(tuple([word, pos_dict.get(tag[0])]))
        return newlist
    
    def lemmatize(pos_data):
        '''
        Performs lemmatization on tokens based on its part of speech tagging 
        '''
        wordnet_lemmatizer = WordNetLemmatizer()
        lemma_rew = " "
        for word, pos in pos_data:
            if not pos:
                lemma = word
                lemma_rew = lemma_rew + " " + lemma
            else:
                lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
                lemma_rew = lemma_rew + " " + lemma
        return lemma_rew
    
    
    df['clean_text'] = df['Text'].apply(lambda x:clean_tweet(x))
    df['POS tagged'] = df['clean_text'].apply(token_stop_pos)
    df['Lemma'] = df['POS tagged'].apply(lemmatize)
    
    return df

In [60]:
data = data_preprocessing(data)

In [61]:
data

,Datetime,Text,Username,Verified,Location,Reply_Count,Retweet_Count,Like_Count,Quote_Count,clean_text,POS tagged,Lemma
0,2022-09-20 12:28:48+00:00,or we could invest in renewables and not worry...,bertytrouser,False,"Wales, United Kingdom",0,0,0,0,or we could invest in renewables and not worry...,"[(could, None), (invest, v), (renewables, n), ...",could invest renewables worry energy connect...
1,2022-09-20 11:46:32+00:00,"Sabine Brink, Blockchain CoE Lead at Shell giv...",Blockchain_Expo,False,Global,1,3,10,1,Sabine Brink Blockchain CoE Lead at Shell givi...,"[(Sabine, n), (Brink, n), (Blockchain, n), (Co...",Sabine Brink Blockchain CoE Lead Shell give ...
2,2022-09-20 03:49:10+00:00,@NadineNakagawa @VancouverSun Shell is working...,moodyangela,False,Vancouver,0,0,0,0,Shell is working with BCNDP and fed Libs to gr...,"[(Shell, n), (working, v), (BCNDP, n), (fed, v...",Shell work BCNDP feed Libs greenwash natural...
3,2022-09-19 20:54:22+00:00,"Top 3 local firms Blue Shell Productions, Baja...",exportbarbados,False,Barbados,1,0,0,0,Top local firms Blue Shell Productions Bajan D...,"[(Top, a), (local, a), (firms, n), (Blue, n), ...",Top local firm Blue Shell Productions Bajan ...
4,2022-09-19 19:04:43+00:00,VIAVIWEB Wallpaper Admin SQL Injection / Shell...,_silvino_,False,Portugal,0,1,0,0,VIAVIWEB Wallpaper Admin SQL Injection Shell U...,"[(VIAVIWEB, n), (Wallpaper, n), (Admin, n), (S...",VIAVIWEB Wallpaper Admin SQL Injection Shell...
...,...,...,...,...,...,...,...,...,...,...,...,...
16066,2017-09-02 20:55:25+00:00,@Java4Dayz @Shell @LiveWIREIntl Mark Twain by ...,Peer_Review1,False,NaN,0,0,0,0,Mark Twain by definitions Clean Energy is fund...,"[(Mark, n), (Twain, n), (definitions, n), (Cle...",Mark Twain definition Clean Energy fundament...
16067,2017-09-02 20:54:11+00:00,@Java4Dayz @Shell @LiveWIREIntl Mark Twain att...,Peer_Review1,False,NaN,0,0,0,0,Mark Twain attached is proof showing energy is...,"[(Mark, n), (Twain, n), (attached, v), (proof,...",Mark Twain attach proof show energy thermody...
16069,2017-09-02 00:34:09+00:00,#entrepeneurs Pak businesses short listed for ...,SocialMediaBoff,False,"Barcelona, Catalunya",0,0,0,0,entrepeneurs Pak businesses short listed for S...,"[(entrepeneurs, n), (Pak, n), (businesses, n),...",entrepeneurs Pak business short list Shell g...
16070,2017-09-02 00:01:02+00:00,Pak businesses short listed for Shell global e...,EINShellNews,False,"Washington, D.C.",0,0,0,0,Pak businesses short listed for Shell global e...,"[(Pak, n), (businesses, n), (short, a), (liste...",Pak business short list Shell global entrepr...


# Getting Sentiments

In [63]:
%%time
def label_tweet(data):
    '''function to predict sentiments using the Vader model
    
    Args: dataframe
    '''
    def vadersentimentanalysis(tweet):
    
        '''
        calculates the sentiment score and returns it
        '''
        analyzer = SentimentIntensityAnalyzer()

        vs = analyzer.polarity_scores(tweet)
        compound = vs['compound']
        return compound
    
    def vader_analysis(compound):
    
        '''
        Maps the sentiment based on sentiment score as 'positive', 'negative' and 'neutral'
        '''
        if compound >= 0.5:
            return 'Positive'
        elif compound <= -0.5 :
            return 'Negative'
        else:
            return 'Neutral'
    
    data['sentiment_score'] = data['Lemma'].apply(vadersentimentanalysis)
    data['sentiment'] = data['sentiment_score'].apply(vader_analysis)
    data = data[['Text','sentiment']]
    
    return data
labelled_data = label_tweet(data)
labelled_data.head(3)

Wall time: 6min 55s


,Text,sentiment
0,or we could invest in renewables and not worry...,Negative
1,"Sabine Brink, Blockchain CoE Lead at Shell giv...",Positive
2,@NadineNakagawa @VancouverSun Shell is working...,Positive
